# Visualisation initiales des donn&eacute;es

In [ ]:
import netCDF4 as nc
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

## Les donn&eacute;es

In [ ]:
data = nc.Dataset('Donnees/Donnees_PIE_Meteo.nc')

Pour se faire une idée du contenu du fichier .netcdf :

In [ ]:
for var in data.variables.values():
    print(var)

### Les dimensions

<ul>
    <li> Temps : 31 échantillons </li>
    <li> Latitude : 361 échantillons </li>
    <li> Longitude : 720 échantillons </li>
</ul>

### Les variables d'int&eacute;r&ecirc;t

<ul>
    <li> 'GP_GDS0_PV' : Le géopotentiel (potentiel gravitationnel au niveau de la tropopause)</li>
    <li> 'POT_GDS0_PV' : La température potentielle (température que le fluide aurait à la pression $P_0=1000hPa$ - moyennant un transport adiabatique)</li>
</ul>

**Ces données sont donc de taille [31,361,720]**

## Visualisation

### A un instant t fixé

In [ ]:
## Variable d'intérêt à afficher
## Géopotentiel : 0  
## Température potentielle : 1
var = 0

## t entier entre 0 et 360 inclus
t = 0


if(var==0):
    var='GP_GDS0_PVL'
else :
    var='POT_GDS0_PVL'

plt.figure(figsize=(10,7))
plt.imshow(data[var][t,:,:])
plt.show()

### Vid&eacute;o 

In [ ]:
def video(nb):
    
    if (nb==0) :
        donnee='GP_GDS0_PVL'
    else :
        donnee='POT_GDS0_PVL'
        
    fig=plt.figure()
        
    frames = []
    fig = plt.figure(figsize=(10,7))
    for i in range(31):
        frames.append([plt.imshow(data[donnee][i,:,:], animated=True)])

    anim = animation.ArtistAnimation(fig, frames, 
                                     interval=150, ## Video plus ou moins rapide/fluide 
                                     blit=True,
                                     repeat=False) ## Reprendre au début à la fin

    return anim

In [ ]:
## Variable d'intérêt à afficher
## Géopotentiel : 0  
## Température potentielle : 1
var = 0

HTML(video(var).to_html5_video())